In [2]:
!pip install --user uproot
!pip install --user pylorentz

  Stored in directory: /tmp/kcollie/.cache/pip/wheels/d2/0b/02/0fc15a343e2879276ca54f7cf26e5815ef7499e890889cbb88
Successfully built pylorentz


In [3]:
import sys
sys.path.append("/eos/home-m/kcollie/.local/lib/python2.7/site-packages")

In [41]:
import uproot 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_curve, roc_auc_score
import xgboost as xgb
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt

from pylorentz import Momentum4
from pylorentz import Position4

In [5]:
# loading the tree
tree = uproot.open("/eos/user/d/dwinterb/SWAN_projects/Masters_CP/MVAFILE_GluGluHToTauTauUncorrelatedDecay_Filtered_tt_2018.root")["ntuple"]
tree2 = uproot.open("/eos/user/d/dwinterb/SWAN_projects/Masters_CP/MVAFILE_VBFHToTauTauUncorrelatedDecay_Filtered_tt_2018.root")["ntuple"]


#These are new trees, added by Danny on 13/10
tree3=uproot.open("/eos/user/d/dwinterb/SWAN_projects/Masters_CP/MVAFILE_AllHiggs_tt.root")["ntuple"]  
#note, instead of the first t we can have e,m, different decay modes

In [11]:
# define what variables are to be read into the dataframe

variables = [ "pi_E_1", "pi_px_1", "pi_py_1", "pi_pz_1", #leading charged pi 4-momentum
              "pi_E_2", "pi_px_2", "pi_py_2", "pi_pz_2", #subleading charged pi 4-momentum
              "ip_x_1", "ip_y_1", "ip_z_1",        #leading impact parameter
              "ip_x_2", "ip_y_2", "ip_z_2",        #subleading impact parameter
                "aco_angle_1"  #acoplanarity angle
             ]

df = tree.pandas.df(variables)

df2 = tree2.pandas.df(variables)

#tree3 should be more comprehensive and complete as it includes all decay types and more
#info for tau->a1 decays. Again we save the same variables for now

#df3 = tree3.pandas.df(variables)

#combine VBF and ggH events

df = pd.concat([df,df2], ignore_index=True)

In [12]:
df.head()

,pi_E_1,pi_px_1,pi_py_1,pi_pz_1,pi_E_2,pi_px_2,pi_py_2,pi_pz_2,ip_x_1,ip_y_1,ip_z_1,ip_x_2,ip_y_2,ip_z_2,aco_angle_1
0,19.151596,-18.843747,0.010897,-3.417199,39.612088,29.067104,9.341342,-25.237687,0.000139,0.001105,-0.000345,-0.002283,-0.000268,-0.002748,2.034394
1,20.922342,-15.508304,11.613920,-7.895206,52.454829,39.534872,-12.412102,32.162451,0.002617,0.005864,0.003067,-0.000048,0.001023,0.000445,-9999.000000
2,127.514681,48.772534,12.352636,-117.169222,77.694478,2.682707,-26.568025,-72.961329,-0.000344,0.003279,0.000161,0.008703,0.016028,-0.005061,0.972370
3,76.979162,-49.683652,22.702982,54.239112,22.436639,17.712197,10.352244,9.082533,-0.003306,0.004030,-0.004681,0.002226,-0.001925,-0.001807,-9999.000000
4,26.184192,5.249920,-12.394966,-22.458754,52.305565,-10.016787,40.401790,-31.673523,-0.000045,0.000480,-0.000170,-0.000894,0.000668,0.000903,5.927902


Before any traning, let's try and reconstruct aco_angle_1 from the low level variable. 
$\mathbf{\lambda}^{\pm}=(0,\mathbf{j}^\pm)$ where $\mathbf{j}^\pm$ is the impact parameter vector defined as the vector btw the primary vertex (PV) and the point of closest approach (PCA).

Then we need to boost $\mathbf{\lambda}^{\pm}$ into the zero momentum frame (ZMF) thus giving $\mathbf{\lambda}^{*\pm}$. I will need to code a function to boost any four vector into the pion's ZMF. 

We will also need $\mathbf{q}^{*\pm}$ are the four-vector for charged pions, boosted in the ZMF. 

After that we will calculate $\mathbf{\lambda}^{*\pm}_{\perp}$ the transverse component of $\mathbf{\lambda}^{*\pm}$ w.r.t. $\mathbf{q}^{*\pm}$. 

Finally, the acoplanary angle $\phi_{CP}$ is definied by 

$\phi_{\mathrm{CP}}=\left\{\begin{array}{ll} \phi^{*} &  \text { if } O^{*} \geq 0 \\  360^{\circ}-\phi^{*} & \text { if } O^{*}<0 \end{array}\right\}$

with $O^{*}=\hat{q}^{*-} \cdot\left(\hat{\lambda}_{\perp}^{*+} \times \hat{\lambda}_{\perp}^{*-}\right)$.


Start with $\mathbf{j}^\pm$, check what we have in terms of available informations.

In [67]:
# Create our 4-vectors in the lab frame
pi_1 = Momentum4(df["pi_E_1"], df["pi_px_1"], df["pi_py_1"], df["pi_pz_1"])
pi_2 = Momentum4(df["pi_E_2"], df["pi_px_2"], df["pi_py_2"], df["pi_pz_2"])

IP1 = Position4(np.zeros(len(df)), df["ip_x_1"], df["ip_y_1"], df["ip_z_1"])
IP2 = Position4(np.zeros(len(df)), df["ip_x_2"], df["ip_y_2"], df["ip_z_2"])

In [68]:
# Create 4-vectors in the ZMF
pi_T4M = pi_1 + pi_2

pi1_ZMF = Pi1.boost_particle(-pi_T4M).transpose()
pi2_ZMF = Pi2.boost_particle(-pi_T4M).transpose()

IP1_ZMF = IP1.boost_particle(-pi_T4M).transpose()
IP2_ZMF = IP2.boost_particle(-pi_T4M).transpose()

In [69]:
# Check we boosted correctly (spatial components should be 0)
P_Total = (Pi1_ZMF+Pi2_ZMF)
P_Total[0]

array([-4.93972337e+01,  1.77635684e-15,  5.32907052e-15, -1.06581410e-14])

In [107]:
# Find the transverse components
IP1_trans = np.cross(IP1_ZMF[1:,:].transpose(), pi1_ZMF[1:, :].transpose())
IP2_trans = np.cross(IP2_ZMF[1:,:].transpose(), pi2_ZMF[1:, :].transpose())

# Normalise the vectors
IP1_trans = IP1_trans/np.linalg.norm(IP1_trans, ord=2, axis=1, keepdims=True)
IP2_trans = IP2_trans/np.linalg.norm(IP2_trans, ord=2, axis=1, keepdims=True)

In [121]:
dot = IP1_trans[:,0]*IP2_trans[:,0]+IP1_trans[:,1]*IP2_trans[:,1]+IP1_trans[:,2]*IP2_trans[:,2]
dot[0]

-0.3234907760118417

In [123]:
Phi_ZMF = np.arccos(dot)

In [125]:
Phi_ZMF.shape

(3684080,)

In [124]:
plt.figure()
plt.hist(Phi_ZMF)
plt.show()

/cvmfs/sft.cern.ch/lcg/views/LCG_97apython3/x86_64-centos7-gcc8-opt/lib/python3.7/site-packages/numpy/lib/histograms.py:824: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
/cvmfs/sft.cern.ch/lcg/views/LCG_97apython3/x86_64-centos7-gcc8-opt/lib/python3.7/site-packages/numpy/lib/histograms.py:825: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)
